In [ ]:
import random
from tqdm.notebook import tqdm
import collections
import pandas as pd

class board_games():

  def __init__(self):
    self.board=[["S", " ", "B", " ", "B", " ", " ", " ", " ", " ", " ", "B", " "],
                [" ", "B", " ", " ", "B", " ", " ", " ", " ", " ", " ", "B", " "],
                [" ", " ", "B", " ", "B", " ", " ", " ", "B", "B", " ", "B", " "],
                [" ", " ", "B", " ", " ", "B", " ", " ", "B", "B", " ", "B", " "],
                ["B", " ", " ", "B", " ", "B", " ", "B", "B", " ", " ", " ", "B"],
                [" ", "B", " ", "B", " ", "B", " ", "B", " ", "B", " ", "B", " "],
                [" ", "B", " ", " ", "B", " ", " ", "B", "B", " ", " ", " ", "B"],
                [" ", " ", "B", " ", " ", " ", " ", "B", "B", " ", " ", " ", "B"],
                [" ", " ", " ", "B", "B", "B", "B", " ", " ", "B", " ", "B", " "],
                [" ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", "B", " "],
                [" ", " ", " ", "B", "B", "B", "B", "B", "B", "B", "B", " ", " "],
                [" ", " ", " ", " ", " ", " ", " ", " ", " ", "B", " ", " ", " "],
                [" ", " ", " ", " ", " ", " ", "B", " ", " ", " ", " ", " ", "E"]]

    self.move={
        "UP" : 1,
        "LEFT" : 2,
        "RIGHT" : 3, 
        "DOWN" :4
        }

    self.penalty_db = []
    self.Q_Dict = {}

    for i in range (13):
      self.Q_Dict[i]={}
      for j in range (13):
        self.Q_Dict[i][j]={}
        for k,v in self.move.items():
          self.Q_Dict[i][j][k]=0


    #Hyperparameters
    self.epsilon = 0.1        #Epsilon-Greedy Policy's Stochasticity
    self.alpha = 0.95         #Moving Avg
    self.gamma = 0.9          #Discount Factor
    
    self.position_wise_penalty = []
    self.bomb_penalty = -100

  def show(self):
    print("""
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} | {} |
    -----------------------------------------------------
    """.format(*[j for i in self.board for j in i]))

  def penalty_initialization(self):
    
    for i in range(13):
      for j in range (13):
        if (self.board[i][j]=='B'):
          
          y_cord,x_cord=i,j
          co_ordinate=[y_cord,x_cord]
         
          self.position_wise_penalty.append(co_ordinate)
    
    return (self.position_wise_penalty)

  def bomb_db(self):
    self.penalty_db = self.penalty_initialization()
    print(self.penalty_db)


  def movement(self, current_pos, dir_act):
    penalty=-1
    ROW = current_pos[0]
    COL = current_pos[1]

    # for key, value in self.move.items():
    
    #   if key == dir:
    move_dir = dir_act

    if(move_dir=='LEFT'):
      ROW = ROW
      COL = COL-1

      if(COL<0 ):
        penalty -=10
        COL +=1
    
      #return (penalty,ROW,COL )

    elif (move_dir == 'UP'):
      ROW = ROW - 1
      COL = COL

      if(ROW < 0):
        penalty -=10
        ROW +=1
    
      #return (penalty,ROW,COL )

    elif (move_dir == 'DOWN'):
      ROW = ROW + 1
      COL = COL

      if(ROW > 12):
        penalty -=10
        ROW -=1
    
      #return (penalty,ROW,COL )
     
    else:
      ROW = ROW
      COL = COL+1

      if(COL >  12):
        penalty -=10
        COL -=1

    for row_key, val in self.Q_Dict.items():
      for col_key , action_value in val.items():
        if([row_key, col_key] == [ROW, COL] ):
          for key, val in action_value.items():
            Action = max( action_value.keys(), key=(lambda k: action_value[k] ))
            r,c = ROW , COL
            Q_Val_Max = self.Q_Dict[r][c][Action]
    
    if ([ROW,COL] in self.penalty_db):
      return Q_Val_Max, -100, [ROW, COL]

    elif ([ROW,COL] == [12,12]):
      return Q_Val_Max, 10 ,[12,12]


    else:
      return Q_Val_Max, penalty,[ROW,COL] 

  
  def exit_status(self, state):
    if state == [12,12]:
      return True
    elif ( state in self.penalty_db):
      return True
    else : 
      return False

  def best_next_state(self, current_state):
    for row_key, val in self.Q_Dict.items():
      for col_key , action_value in val.items():
        if([row_key, col_key] == current_state ):
          if (all(value==0 for value in action_value.values())):
            Action = random.choice (['UP','DOWN','LEFT','RIGHT'])
          else:
            Action = max( action_value.keys(), key=(lambda k: action_value[k] ))

    return(Action)

  def return_action_value (self, state, action):
    for row_key, val in self.Q_Dict.items():
      for col_key , action_value in val.items():
        if([row_key, col_key] == state):
          for key, val in action_value.items():
            if (key==action):
              return(val)

  def reverse_dictionary(self, val):
    for k, v in self.move.items():
      if v == val:
        return(k)

  def Q_Table_Update(self, current_state, action, upd_value):
    for row_key, val in self.Q_Dict.items():
      for col_key , action_value in val.items():
        if([row_key, col_key] == current_state):
          for key, val in action_value.items():
            if (key==action):
              self.Q_Dict[row_key][col_key][key] = upd_value

 
  def Q_Learning(self):
    Initial_Point = [[0,0]] #Training from random point is also possible.

    for item in Initial_Point:

      for i in tqdm(range (10000)):

        row = item[0]
        col = item[1]
        action = random.randint(1,4)
        exit_zone = False
        current_state = [row, col]

        reward = -1

        while not exit_zone:
          if (random.uniform(0,1) < self.epsilon):
            act = random.randint(1,4)
            action = self.reverse_dictionary(act)
          else:
            action = self.best_next_state(current_state)

          Q_Max_reward, reward,  next_state = self.movement (current_state, action)

          exit_zone = self.exit_status(current_state)

          old_value = self.return_action_value(current_state, action)

          
          new_value = old_value + self.alpha * (reward + self.gamma*Q_Max_reward - old_value)

          if not self.exit_status(current_state):
            self.Q_Table_Update(current_state, action, new_value)
            current_state = next_state

        if (i% 20000 == 0):
          print('Iteration no {}'.format(i))

    new_dict = {}
    for row_key, val in self.Q_Dict.items():
      for col_key , v in val.items():
        print(row_key,col_key, ':', v)
        new_key = str(row_key)+','+str(col_key)
        new_dict[new_key] = {}
        new_dict[new_key]=  v
    
    sort_new_dict = collections.OrderedDict(sorted(new_dict.items()))
    df = pd.DataFrame.from_dict(sort_new_dict, orient='index')
    df.to_excel (r'Learned_ActionValue_dataframe.xlsx', index = True, header=True)

    print(df)

    self.test()

  def test(self):
    path = []
    count =0
    cur_state =[0,0]
    while cur_state != [12,12]:
      path.append(cur_state)
      act = self.best_next_state(cur_state)
      q_val, penalty, next_state =self. movement(cur_state,act )
      #print (act, next_state,'->', q_val )   #This Line Helped me to debug
      cur_state = next_state
      if (penalty == -100):
        count += 1
    print(path)
    print("You Have Died {} times".format(count))
           



rl_board=board_games()
rl_board.show()
penalty = rl_board.penalty_initialization()

rl_board.bomb_db()

rl_board.Q_Learning()
#rl_board.test()

In [ ]:
print()

(0,2) means 0th row 2nd column

In [ ]:
import collections
import pandas as pd


my_dict={}
new_dict_1 = {}
my_dict[1] = {}
my_dict[2] = {}
my_dict[1][1]={}
my_dict[2][3]={}
my_dict[1][1]['up'] = 10
my_dict[1][1]['down'] = 30
my_dict[1][1]['left'] = 20
my_dict[1][1]['right'] = 60

my_dict[2][3]['up'] = .1
my_dict[2][3]['down'] = -1
my_dict[2][3]['left'] = -10
my_dict[2][3]['right'] = 1


upd_value = 100
current_state = [2,3]
#action = 'right'
# print(my_dict)
for row_key, val in my_dict.items():
  for k , v in val.items():
    print(row_key,k, ':', v)
    
    new_key = str(row_key)+','+str(k)
    new_dict_1[new_key] = {}
    new_dict_1[new_key]=  v

sort_new_dict = collections.OrderedDict(sorted(new_dict_1.items()))
df = pd.DataFrame.from_dict(sort_new_dict, orient='index')
print(df)
df.to_excel (r'export_dataframe.xlsx', index = True, header=True)

print(sort_new_dict)

# print (r, c, Q_Val_Max)


1 1 : {'up': 10, 'down': 30, 'left': 20, 'right': 60}
2 3 : {'up': 0.1, 'down': -1, 'left': -10, 'right': 1}
       up  down  left  right
1,1  10.0    30    20     60
2,3   0.1    -1   -10      1
OrderedDict([('1,1', {'up': 10, 'down': 30, 'left': 20, 'right': 60}), ('2,3', {'up': 0.1, 'down': -1, 'left': -10, 'right': 1})])


In [ ]:
my_list = [[0,0],[7,3], [0,6],[7,10]]

for item in my_list:
  print(item[0])

0
7
0
7
